In [1]:
from pathlib import Path

base = Path(r"C:\Users\Gresa Calliku\Documents\20250518_GC_RescU\RescU")

# Define folder structure
folders = [
    "data/geospatial",
    "data/satellite",
    "data/scripts",
    "models",
    "notebooks"
]

# Create folders
for folder in folders:
    (base / folder).mkdir(parents=True, exist_ok=True)

# Create ReadMe.md
(base / "ReadMe.md").touch()


In [2]:
from pathlib import Path

# Base path where structure should be created
base = Path(r"C:\Users\Gresa Calliku\Documents\20250518_GC_RescU\RescU")

# Folder structure
folders = [
    "data/geospatial",
    "data/satellite",
    "data/scripts",
    "models",
    "notebooks"
]

# Create folders and add ReadMe.md in each
for folder in folders:
    folder_path = base / folder
    folder_path.mkdir(parents=True, exist_ok=True)
    readme_file = folder_path / "ReadMe.md"
    readme_file.write_text(f"# {folder_path.name}\n\nThis folder contains {folder_path.name} files.\n")

# Create top-level ReadMe.md
top_readme = base / "ReadMe.md"
top_readme.write_text("# RescU Project\n\nThis repository contains geospatial data, models, and notebooks related to rescue simulation and accessibility analysis.")


137

In [4]:
!conda install -c conda-forge osmnx -y

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: done

## Package Plan ##

  environment location: C:\Users\Gresa Calliku\miniconda3

  added / updated specs:
    - osmnx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    affine-2.4.0               |     pyhd8ed1ab_1          19 KB  conda-forge
    blas-1.0                   |              mkl           1 KB  conda-forge
    blosc-1.21.3               |       h6c2663c_0          86 KB
    boost-cpp-1.82.0           |       h59b6b97_2          11 KB
    bottleneck-1.4.2           |  py313h2cb717b_0         145 KB
    branca-0.8.1               |     pyhd8ed1ab_0          29 KB  conda-forge
    brotli-1.0.9               |       hcfcfb64_8          19 KB  conda-forge
    brotli-bin-1.0.9           |       hcfcfb64_8          22 KB  conda-forge
    cairo-1.16.0               |       hc68a040_5         2.3 MB
    cfi

In [6]:
# this script can be called download_osm.py
import osmnx as ox

# Define your place of interest
place = "Berlin, Germany"

# Download building footprints
buildings = ox.geometries_from_place(place, tags={"building": True})

# Download road network
roads = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads, nodes=False)

# Download fire station locations
fire_stations = ox.geometries_from_place(place, tags={"emergency": "fire_station"})

# Save as GeoPackage for reuse
buildings.to_file("berlin_buildings.gpkg", layer='buildings', driver="GPKG")
roads_gdf.to_file("berlin_roads.gpkg", layer='roads', driver="GPKG")
fire_stations.to_file("berlin_fire_stations.gpkg", layer='fire_stations', driver="GPKG")

AttributeError: module 'osmnx' has no attribute 'geometries_from_place'

In [7]:
import osmnx as ox
print(ox.__version__)

2.0.3


In [8]:
import osmnx as ox

# Enable logging (optional)
ox.config(log_console=True)

# Define your area of interest
place = "Berlin, Germany"

# Define tags (e.g., all buildings)
tags = {"building": True}

# Get building footprints as GeoDataFrame
buildings = ox.features_from_place(place, tags=tags)

# Get drivable street network as graph
roads = ox.graph_from_place(place, network_type="drive")


AttributeError: module 'osmnx' has no attribute 'config'

In [9]:
import osmnx as ox
import logging

# Optional: enable logging
logging.basicConfig(level=logging.INFO)

# Define area of interest
place = "Berlin, Germany"
tags = {"building": True}

# Download building footprints
buildings = ox.features_from_place(place, tags=tags)

# Download road network
roads = ox.graph_from_place(place, network_type="drive")


MemoryError: Unable to allocate 5.91 GiB for an array with shape (1320, 600937) and data type object

In [ ]:
# download_osm.py

import osmnx as ox

# Define your place of interest
place = "Berlin, Germany"

# Download building footprints
buildings = ox.features_from_place(place, tags={"building": True})

# Download road network and convert to GeoDataFrame
roads_graph = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads_graph, nodes=False)

# Download fire station locations
fire_stations = ox.features_from_place(place, tags={"emergency": "fire_station"})

# Save to GeoPackages
buildings.to_file("berlin_buildings.gpkg", layer='buildings', driver="GPKG")
roads_gdf.to_file("berlin_roads.gpkg", layer='roads', driver="GPKG")
fire_stations.to_file("berlin_fire_stations.gpkg", layer='fire_stations', driver="GPKG")


In [10]:
%run download_osm.py

Exception: File `'download_osm.py'` not found.

In [ ]:
# download_sentinel.py

from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date
import osmnx as ox

# Sentinel credentials (replace with your own Copernicus credentials)
username = 'your_username'
password = 'your_password'
api_url = 'https://scihub.copernicus.eu/dhus'

# Connect to API
api = SentinelAPI(username, password, api_url)

# Get bounding geometry for Berlin
area_gdf = ox.geocode_to_gdf("Berlin, Germany")
area_geom = area_gdf.geometry.values[0]
area_wkt = area_geom.to_wkt()  # Convert to WKT for the API

# Query Sentinel-2 products
products = api.query(area_wkt,
                     date=('20240101', '20240115'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

# Check and download the first available product
if products:
    for product_id, product_info in products.items():
        print(f"Downloading: {product_info['title']}")
        api.download(product_id)
        break
else:
    print("No products found for the specified query.")


In [11]:
import osmnx as ox

# Smaller area to avoid memory errors
place = "Mitte, Berlin, Germany"
tags = {"building": True}

# Download building footprints
buildings = ox.features_from_place(place, tags=tags)

# Download drivable street network
roads_graph = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads_graph, nodes=False)

# Download fire stations
fire_stations = ox.features_from_place(place, tags={"emergency": "fire_station"})

# Save to GeoPackage (within a local folder structure if desired)
buildings.to_file("mitte_buildings.gpkg", layer="buildings", driver="GPKG")
roads_gdf.to_file("mitte_roads.gpkg", layer="roads", driver="GPKG")
fire_stations.to_file("mitte_fire_stations.gpkg", layer="fire_stations", driver="GPKG")



InsufficientResponseError: No matching features. Check query location, tags, and log.

In [13]:
import osmnx as ox
import geopandas as gpd

place = "Mitte, Berlin, Germany"
tags = {"building": True}

# Download buildings
buildings = ox.features_from_place(place, tags=tags)

# Download streets
roads_graph = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads_graph, nodes=False)

# Try downloading fire stations
try:
    fire_stations = ox.features_from_place(place, tags={"emergency": "fire_station"})
except ox._errors.InsufficientResponseError:
    print("⚠️ No fire stations found — continuing with empty GeoDataFrame.")
    fire_stations = gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

# Save all layers
buildings.to_file("mitte_buildings.gpkg", layer="buildings", driver="GPKG")
roads_gdf.to_file("mitte_roads.gpkg", layer="roads", driver="GPKG")
fire_stations.to_file("mitte_fire_stations.gpkg", layer="fire_stations", driver="GPKG")


⚠️ No fire stations found — continuing with empty GeoDataFrame.


C:\Users\Gresa Calliku\miniconda3\Lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()
INFO:pyogrio._io:Created 24,838 records
INFO:pyogrio._io:Created 4,644 records
INFO:pyogrio._io:Created 0 records


In [14]:
import osmnx as ox
import geopandas as gpd

# Define smaller area to stay within memory limits
place = "Mitte, Berlin, Germany"
tags = {"building": True}

# Download building footprints
buildings = ox.features_from_place(place, tags=tags)

# Download road network and convert to GeoDataFrame
roads_graph = ox.graph_from_place(place, network_type='drive')
roads_gdf = ox.graph_to_gdfs(roads_graph, nodes=False)

# Try downloading fire station locations
try:
    fire_stations = ox.features_from_place(place, tags={"emergency": "fire_station"})
except ox._errors.InsufficientResponseError:
    print("⚠️ No fire stations found — creating empty GeoDataFrame.")
    fire_stations = gpd.GeoDataFrame(geometry=[], crs="EPSG:4326")

# Save to GeoJSON files
buildings.to_file("mitte_buildings.geojson", driver="GeoJSON")
roads_gdf.to_file("mitte_roads.geojson", driver="GeoJSON")
fire_stations.to_file("mitte_fire_stations.geojson", driver="GeoJSON")


⚠️ No fire stations found — creating empty GeoDataFrame.


INFO:pyogrio._io:Created 24,838 records
INFO:pyogrio._io:Created 4,644 records
INFO:pyogrio._io:Created 0 records


In [16]:
!pip install sentinelsat

In [17]:
# download_sentinel.py

from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date
import osmnx as ox

# --- Configuration ---
username = 'your_username'  # Replace with your Copernicus SciHub username
password = 'your_password'  # Replace with your password
api_url = 'https://scihub.copernicus.eu/dhus'

# --- Connect to API ---
api = SentinelAPI(username, password, api_url)

# --- Define area using OSMnx ---
place = "Berlin, Germany"
area_gdf = ox.geocode_to_gdf(place)
area_geom = area_gdf.geometry.values[0]
area_wkt = geojson_to_wkt(area_geom.__geo_interface__)  # convert to WKT

# Optional: Save area to GeoJSON for reuse
area_gdf.to_file("berlin_area.geojson", driver="GeoJSON")

# --- Query Sentinel-2 products ---
products = api.query(area_wkt,
                     date=('20240101', '20240115'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

# --- Download first result if available ---
if products:
    for product_id, product_info in products.items():
        print(f"Downloading {product_info['title']}")
        api.download(product_id)
        break
else:
    print("⚠️ No Sentinel-2 products found for the given query.")


INFO:pyogrio._io:Created 1 records


ConnectTimeout: HTTPSConnectionPool(host='scihub.copernicus.eu', port=443): Max retries exceeded with url: /dhus/search?format=json&rows=100&start=0&q=beginPosition%3A%5B%222024-01-01T00%3A00%3A00Z%22+TO+%222024-01-15T00%3A00%3A00Z%22%5D+cloudcoverpercentage%3A%5B%220%22+TO+%2210%22%5D+platformname%3A%22Sentinel-2%22+footprint%3A%22Intersects%28MULTIPOLYGON%28%28%2813.0883+52.4196%2C13.0902+52.4136%2C13.0902+52.4134%2C13.0902+52.4127%2C13.0903+52.4122%2C13.0908+52.4116%2C13.0914+52.4114%2C13.0921+52.4112%2C13.0924+52.4111%2C13.0934+52.4105%2C13.0974+52.4094%2C13.0980+52.4099%2C13.0980+52.4099%2C13.0983+52.4101%2C13.0984+52.4101%2C13.0986+52.4103%2C13.0986+52.4103%2C13.0987+52.4103%2C13.0985+52.4104%2C13.0986+52.4104%2C13.0986+52.4104%2C13.0986+52.4104%2C13.0986+52.4104%2C13.0986+52.4104%2C13.0986+52.4104%2C13.0986+52.4104%2C13.0985+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13.0986+52.4105%2C13

In [18]:
from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date
import osmnx as ox
from shapely.geometry import box

# Sentinel Hub credentials
username = 'your_username'
password = 'your_password'
api_url = 'https://scihub.copernicus.eu/dhus'

# Connect to Sentinel API
api = SentinelAPI(username, password, api_url)

# Define a smaller area: Berlin-Mitte
place = "Mitte, Berlin, Germany"
area_gdf = ox.geocode_to_gdf(place)
area_geom = area_gdf.geometry.values[0]

# Option 1: Use bounding box for performance and shorter query
bbox_geom = box(*area_geom.bounds)
area_wkt = geojson_to_wkt(bbox_geom.__geo_interface__)

# Save bounding box as GeoJSON (optional)
area_gdf.to_file("berlin_mitte.geojson", driver="GeoJSON")

# Query Sentinel-2 products
products = api.query(area_wkt,
                     date=('20240101', '20240115'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 10))

# Download the first result
if products:
    for product_id, product_info in products.items():
        print(f"Downloading {product_info['title']}")
        api.download(product_id)
        break
else:
    print("⚠️ No Sentinel-2 products found for Mitte within this date and cloud cover.")


INFO:pyogrio._io:Created 1 records


ConnectTimeout: HTTPSConnectionPool(host='scihub.copernicus.eu', port=443): Max retries exceeded with url: /dhus/search?format=json&rows=100&start=0&q=beginPosition%3A%5B%222024-01-01T00%3A00%3A00Z%22+TO+%222024-01-15T00%3A00%3A00Z%22%5D+cloudcoverpercentage%3A%5B%220%22+TO+%2210%22%5D+platformname%3A%22Sentinel-2%22+footprint%3A%22Intersects%28POLYGON%28%2813.4294+52.4987%2C13.4294+52.5677%2C13.3015+52.5677%2C13.3015+52.4987%2C13.4294+52.4987%29%29%29%22 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000023A7D83EFD0>, 'Connection to scihub.copernicus.eu timed out. (connect timeout=None)'))

In [19]:
# download_sentinel.py

from sentinelsat import SentinelAPI, geojson_to_wkt
from datetime import date
import osmnx as ox
from shapely.geometry import box
import geopandas as gpd
import requests
from requests.exceptions import ConnectTimeout, ReadTimeout

# --- User credentials (replace with your actual account info) ---
USERNAME = 'your_username'
PASSWORD = 'your_password'
API_URL = 'https://scihub.copernicus.eu/dhus'

# --- Setup Sentinel API client ---
api = SentinelAPI(USERNAME, PASSWORD, API_URL)

# --- Get bounding box of Berlin-Mitte using OSMnx ---
place = "Mitte, Berlin, Germany"
area_gdf = ox.geocode_to_gdf(place)
area_geom = area_gdf.geometry.values[0]

# Use bounding box to reduce query string size
bbox_geom = box(*area_geom.bounds)
area_wkt = geojson_to_wkt(bbox_geom.__geo_interface__)

# Save query area to GeoJSON
gpd.GeoDataFrame(geometry=[bbox_geom], crs="EPSG:4326").to_file("berlin_mitte_bbox.geojson", driver="GeoJSON")

# --- Query Sentinel-2 products ---
try:
    products = api.query(
        area_wkt,
        date=('20240101', '20240115'),
        platformname='Sentinel-2',
        cloudcoverpercentage=(0, 10)
    )
    if products:
        for product_id, product_info in products.items():
            print(f"✅ Found product: {product_info['title']}")
            api.download(product_id)
            break
    else:
        print("⚠️ No Sentinel-2 products found for the specified query.")
except (ConnectTimeout, ReadTimeout) as e:
    print("❌ Connection to Copernicus SciHub timed out. Please try again later.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")


INFO:pyogrio._io:Created 1 records


❌ Connection to Copernicus SciHub timed out. Please try again later.


In [1]:
gitignore_path = ".gitignore"

# Define the folders you want to ignore
lines_to_add = [
    "RescU/data/geospatial/",
    "RescU/data/satellite/"
]

# Append to .gitignore (create it if it doesn't exist)
with open(gitignore_path, "a+", encoding="utf-8") as f:
    f.seek(0)
    existing = f.read().splitlines()
    for line in lines_to_add:
        if line not in existing:
            f.write(f"{line}\n")

print(".gitignore updated.")


.gitignore updated.
